# Turbulence Modeling Example

This notebook demonstrates various turbulence modeling approaches in PHASTA, including RANS, LES, and hybrid methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phasta import FlowSolver, FlowConfig, Mesh, TurbulenceModelConfig

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Problem Setup

We'll simulate flow over a backward-facing step with the following parameters:
- Step height: 1 unit
- Channel height: 3 units
- Channel length: 20 units
- Inlet velocity: 1 unit/s
- Reynolds number: 5000

In [ ]:
# Create base configuration
config = FlowConfig()
config.domain = {
    'step_height': 1.0,
    'channel_height': 3.0,
    'channel_length': 20.0,
    'mesh_size': 0.05
}

config.flow = {
    'inlet_velocity': 1.0,
    'reynolds_number': 5000,
    'fluid_density': 1.0,
    'fluid_viscosity': 0.0002
}

config.boundary_conditions = {
    'inlet': 'velocity',
    'outlet': 'pressure',
    'walls': 'no-slip'
}

## RANS Models

Let's compare different RANS models: k-ε, k-ω, and Spalart-Allmaras.

In [ ]:
# Create mesh
mesh = Mesh.generate_backward_step_2d(
    step_height=config.domain['step_height'],
    channel_height=config.domain['channel_height'],
    channel_length=config.domain['channel_length'],
    nx=400,
    ny=60
)

# Plot mesh
plt.figure(figsize=(12, 4))
mesh.plot()
plt.title('Backward-Facing Step Mesh')
plt.show()

In [ ]:
# Run simulations with different RANS models
rans_models = ['k-epsilon', 'k-omega', 'spalart-allmaras']
rans_results = {}

for model in rans_models:
    # Configure turbulence model
    turb_config = TurbulenceModelConfig()
    turb_config.model = model
    turb_config.wall_treatment = 'standard'
    
    # Create and run solver
    solver = FlowSolver(config, mesh, turbulence_config=turb_config)
    rans_results[model] = solver.solve(max_iterations=2000, convergence_tolerance=1e-6)
    
    print(f"\n{model.upper()} Model Results:")
    print(f"Converged: {rans_results[model].converged}")
    print(f"Final residual: {rans_results[model].final_residual}")
    print(f"Iterations: {rans_results[model].iterations}")

## Compare RANS Results

Plot velocity profiles and turbulent quantities for different RANS models.

In [ ]:
# Plot velocity profiles
plt.figure(figsize=(15, 5))

# Plot at x = 5 (after step)
plt.subplot(121)
x_pos = 5.0
for model in rans_models:
    y, u = rans_results[model].get_velocity_profile(x_pos)
    plt.plot(u, y, label=model)
plt.xlabel('Velocity')
plt.ylabel('y')
plt.title(f'Velocity Profile at x = {x_pos}')
plt.legend()
plt.grid(True)

# Plot turbulent kinetic energy
plt.subplot(122)
for model in rans_models:
    y, k = rans_results[model].get_turbulent_kinetic_energy(x_pos)
    plt.plot(k, y, label=model)
plt.xlabel('Turbulent Kinetic Energy')
plt.ylabel('y')
plt.title(f'TKE Profile at x = {x_pos}')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## LES Simulation

Now let's run a Large Eddy Simulation with the Smagorinsky model.

In [ ]:
# Configure LES
les_config = TurbulenceModelConfig()
les_config.model = 'smagorinsky'
les_config.smagorinsky_constant = 0.1
les_config.wall_treatment = 'wall-modeled'

# Create and run solver
les_solver = FlowSolver(config, mesh, turbulence_config=les_config)
les_results = les_solver.solve(max_iterations=2000, convergence_tolerance=1e-6)

print("\nLES Results:")
print(f"Converged: {les_results.converged}")
print(f"Final residual: {les_results.final_residual}")
print(f"Iterations: {les_results.iterations}")

## Compare RANS and LES Results

Plot instantaneous and time-averaged results from LES compared to RANS.

In [ ]:
plt.figure(figsize=(15, 10))

# Plot instantaneous velocity field
plt.subplot(221)
les_results.plot_velocity_magnitude()
plt.title('LES: Instantaneous Velocity')
plt.colorbar(label='Velocity')

# Plot time-averaged velocity field
plt.subplot(222)
les_results.plot_time_averaged_velocity()
plt.title('LES: Time-Averaged Velocity')
plt.colorbar(label='Velocity')

# Plot velocity profiles
plt.subplot(223)
x_pos = 5.0
y_les, u_les = les_results.get_time_averaged_velocity_profile(x_pos)
plt.plot(u_les, y_les, 'k-', label='LES (time-averaged)')
for model in rans_models:
    y, u = rans_results[model].get_velocity_profile(x_pos)
    plt.plot(u, y, '--', label=f'{model} (RANS)')
plt.xlabel('Velocity')
plt.ylabel('y')
plt.title(f'Velocity Profile at x = {x_pos}')
plt.legend()
plt.grid(True)

# Plot turbulent quantities
plt.subplot(224)
y_les, k_les = les_results.get_time_averaged_tke(x_pos)
plt.plot(k_les, y_les, 'k-', label='LES (time-averaged)')
for model in rans_models:
    y, k = rans_results[model].get_turbulent_kinetic_energy(x_pos)
    plt.plot(k, y, '--', label=f'{model} (RANS)')
plt.xlabel('Turbulent Kinetic Energy')
plt.ylabel('y')
plt.title(f'TKE Profile at x = {x_pos}')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Hybrid RANS/LES

Finally, let's try a hybrid RANS/LES approach using Detached Eddy Simulation (DES).

In [ ]:
# Configure DES
des_config = TurbulenceModelConfig()
des_config.model = 'des'
des_config.base_model = 'spalart-allmaras'
des_config.wall_treatment = 'hybrid'

# Create and run solver
des_solver = FlowSolver(config, mesh, turbulence_config=des_config)
des_results = des_solver.solve(max_iterations=2000, convergence_tolerance=1e-6)

print("\nDES Results:")
print(f"Converged: {des_results.converged}")
print(f"Final residual: {des_results.final_residual}")
print(f"Iterations: {des_results.iterations}")

## Compare All Approaches

Plot velocity profiles and turbulent quantities for all modeling approaches.

In [ ]:
plt.figure(figsize=(15, 5))

# Plot velocity profiles
plt.subplot(121)
x_pos = 5.0
y_des, u_des = des_results.get_time_averaged_velocity_profile(x_pos)
plt.plot(u_des, y_des, 'k-', label='DES')
y_les, u_les = les_results.get_time_averaged_velocity_profile(x_pos)
plt.plot(u_les, y_les, 'b--', label='LES')
for model in rans_models:
    y, u = rans_results[model].get_velocity_profile(x_pos)
    plt.plot(u, y, ':', label=f'{model} (RANS)')
plt.xlabel('Velocity')
plt.ylabel('y')
plt.title(f'Velocity Profile at x = {x_pos}')
plt.legend()
plt.grid(True)

# Plot turbulent quantities
plt.subplot(122)
y_des, k_des = des_results.get_time_averaged_tke(x_pos)
plt.plot(k_des, y_des, 'k-', label='DES')
y_les, k_les = les_results.get_time_averaged_tke(x_pos)
plt.plot(k_les, y_les, 'b--', label='LES')
for model in rans_models:
    y, k = rans_results[model].get_turbulent_kinetic_energy(x_pos)
    plt.plot(k, y, ':', label=f'{model} (RANS)')
plt.xlabel('Turbulent Kinetic Energy')
plt.ylabel('y')
plt.title(f'TKE Profile at x = {x_pos}')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Exercises

1. Try different RANS models and compare their performance
2. Experiment with LES parameters (filter width, Smagorinsky constant)
3. Modify the DES configuration and observe the blending behavior
4. Compare computational cost and accuracy of different approaches

## Next Steps

- Try the heat transfer example
- Explore multi-phase flow simulation
- Learn about chemical reactions